In [ ]:
import json
import os
import scipy.ndimage
import numpy as np
import matplotlib.path as mpltPath
from matplotlib.path import Path
from random import shuffle
from openslide import open_slide, ImageSlide
import scipy.io as sio
import pdb


In [ ]:
#File paths
slide_path = '/mys3bucket/TCGA_LUSC'
slides = os.listdir(slide_path)

def split(data):
    N = len(data)
    trn_idx = int(np.ceil(0.7*N))
    dev_idx = int(np.ceil(0.2*(N)))
    
    train = data[:trn_idx]
    dev = data[trn_idx:trn_idx+dev_idx]
    test = data[trn_idx+dev_idx:]
    
    return train,dev,test


def get_mask(coords):
    coords.sort(key=lambda x: x[0],reverse=True)
    xmin,xmax = coords[-1][0],coords[0][0]
    coords.sort(key=lambda x: x[1],reverse=True)
    ymin,ymax = coords[-1][1],coords[0][1]
    maximum = max(xmax,ymax)
    minimum = min(xmin,ymin)
    x, y = np.meshgrid(np.arange(minimum,maximum), np.arange(minimum,maximum))
    print("Done generating meshgrid!")
    x, y = x.flatten(), y.flatten()
    points = np.vstack((x,y)).T
    p = Path(coords)
    grid = p.contains_points(points)
    mask = grid.reshape((maximum-minimum),(maximum-minimum))
    x_coords,y_coords = np.nonzero(mask)
    return x_coords,y_coords
    
def read_patches(x_coords,y_coords,slide_src,label):
    gen_data = []
    image = open_slide(slide_src)
    sample_idxs = np.random.choice(np.arange(len(x_coords)),100)
    
    for i in sample_idxs:
        patch = image.read_region((x_coords[i],y_coords[i]),0,(256,256))
        patch = patch.convert("RGB")
        patch = np.array(patch)
        gen_data.append([patch,label])
        
        #Code to save patches as images
        #outfile = os.path.join(slide_dest,"patch_"+str(i)+".jpg")
        #patch.save(outfile,'JPEG')
        #g.write(("patch_"+str(count)+","+str(x_coords[i])+","+str(y_coords[i])+"\n"))
        
    image.close()
    print("Generated patches!")
    return gen_data

def get_slide_path(slideID):
    for slide in slides:
        if str(slideID) == str(slide.split('_')[0]):
            return os.path.join(slide_path,slide)
    return -1

def get_random_polygon(shape):
    if len(shape)>1:
        return shape
    return -1

def generate_data(data):
    DATA = []
    
    for annotation in data:
        slide = annotation['slideId']
        shape = annotation['shape']
        label = annotation['annotationSubstanceId']
        polygon = get_random_polygon(shape)
        slide_src = get_slide_path(slide)
        
        if slide_src == -1:
            print(str(slide)+" file does not exist")
            continue
        if polygon == -1:
            print(str(slide)+" has only point annotation")
            continue

        coords = [tuple(x) for x in polygon]
        x_coords,y_coords = get_mask(coords)
        DATA.extend(read_patches(x_coords,y_coords,slide_src,label))
        print("*****************************************************")
    return np.asarray(DATA)

In [ ]:
#Shuffle the data
f = open("/mys3bucket/Annotations/annotations.txt", encoding="utf-8")
data = json.loads(f.read())
f.close()


shuffle(data)
train,dev,test = split(data)

train_dataset = generate_data(train)
print("Train data generated!")
dev_dataset = generate_data(dev)
print("Dev data generated!")
test_dataset = generate_data(test)
print("Test data generated! ")

# Save to numpy files
train_outfile = "train.npy"
dev_outfile = "dev.npy"
test_outfile = "test.npy"

np.save(train_outfile,train_dataset)
np.save(dev_outfile,dev_dataset)
np.save(test_outfile,test_dataset)

            